### Import Dependencies

In [8]:
from fastmcp import Client

from pydantic import BaseModel, Field

from qdrant_client import QdrantClient
from qdrant_client.models import Prefetch, Filter, FieldCondition, MatchText, FusionQuery, MatchValue

from langsmith import traceable, get_current_run_tree

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

from langchain_core.messages import AIMessage, ToolMessage, convert_to_openai_messages

from jinja2 import Template
from typing import Literal, Dict, Any, Annotated, List, Optional
from IPython.display import Image, display
from operator import add
from openai import OpenAI

from utils.utils import format_ai_message

import openai

import random
import ast
import inspect
import instructor
import json

import psycopg2
from psycopg2.extras import RealDictCursor
import numpy as np
from langchain_core.messages import AnyMessage
from langchain_core.messages import AIMessage, ToolMessage, convert_to_openai_messages

### Add to Shopping Cart Tool

In [17]:
items = [
    {
        "product_id": "B0BLVNZNWV",
        "quantity": 2,
    },
    {
        "product_id": "B0BM4L3J9Q",
        "quantity": 4
    }
]

In [18]:
from __future__ import annotations

from typing import Any
import numpy as np
import psycopg2
from psycopg2.extras import RealDictCursor

from qdrant_client import QdrantClient
from qdrant_client.http.models import (
    Prefetch,
    Filter,
    FieldCondition,
    MatchValue,
    FusionQuery,
)

def add_to_shopping_cart(items: list[dict], user_id: str, cart_id: str) -> str:
    """
    Add a list of provided items to the shopping cart.

    items: [{"product_id": "...", "quantity": 1}, ...]
    """

    # Create clients once (not inside the loop)
    qdrant_client = QdrantClient(url="http://localhost:6333")

    conn = psycopg2.connect(
        host="localhost",
        port=5433,
        database="tools_database",
        user="langgraph_user",
        password="langgraph_password",
    )
    conn.autocommit = True

    # Prepare once
    dummy_vector = np.zeros(1536, dtype=float).tolist()
    collection_name = "Amazon-items-collection-01-hybrid-search"
    embedding_name = "text-embedding-3-small"

    with conn.cursor(cursor_factory=RealDictCursor) as cursor:
        for item in items:
            # Basic validation
            if "product_id" not in item or "quantity" not in item:
                raise ValueError("Each item must include 'product_id' and 'quantity'.")

            product_id = item["product_id"]
            quantity = item["quantity"]

            if quantity is None or int(quantity) <= 0:
                raise ValueError(f"Quantity must be a positive integer. Got: {quantity}")

            # Qdrant lookup with guard against empty result
            result = qdrant_client.query_points(
                collection_name=collection_name,
                prefetch=[
                    Prefetch(
                        query=dummy_vector,
                        filter=Filter(
                            must=[
                                FieldCondition(
                                    key="parent_asin",
                                    match=MatchValue(value=str(product_id)),
                                )
                            ]
                        ),
                        using=embedding_name,
                        limit=20,
                    )
                ],
                query=FusionQuery(fusion="rrf"),
                limit=1,
            )

            if not result.points:
                # Clear error instead of IndexError
                raise ValueError(
                    f"No product found in Qdrant for parent_asin={product_id} "
                    f"in collection '{collection_name}'."
                )

            payload: dict[str, Any] = result.points[0].payload or {}

            product_image_url = payload.get("image")
            price = payload.get("price")

            # If your DB column is NOT NULL, decide how to handle missing price
            if price is None:
                raise ValueError(f"Missing 'price' in Qdrant payload for product_id={product_id}")

            currency = "USD"

            # Check if item already exists
            check_query = """
                SELECT id, quantity, price
                FROM shopping_carts.shopping_cart_items
                WHERE user_id = %s AND shopping_cart_id = %s AND product_id = %s
            """
            cursor.execute(check_query, (user_id, cart_id, str(product_id)))
            existing_item = cursor.fetchone()

            if existing_item:
                new_quantity = int(existing_item["quantity"]) + int(quantity)

                update_query = """
                    UPDATE shopping_carts.shopping_cart_items
                    SET
                        quantity = %s,
                        price = %s,
                        currency = %s,
                        product_image_url = COALESCE(%s, product_image_url)
                    WHERE user_id = %s AND shopping_cart_id = %s AND product_id = %s
                    RETURNING id, quantity, price
                """
                cursor.execute(
                    update_query,
                    (
                        new_quantity,
                        price,
                        currency,
                        product_image_url,
                        user_id,
                        cart_id,
                        str(product_id),
                    ),
                )
            else:
                insert_query = """
                    INSERT INTO shopping_carts.shopping_cart_items (
                        user_id, shopping_cart_id, product_id,
                        price, quantity, currency, product_image_url
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s)
                    RETURNING id, quantity, price
                """
                cursor.execute(
                    insert_query,
                    (
                        user_id,
                        cart_id,
                        str(product_id),
                        price,
                        int(quantity),
                        currency,
                        product_image_url,
                    ),
                )

    return f"Added {items} to the shopping cart."

In [19]:
add_to_shopping_cart(items, "abc", "cde")

"Added [{'product_id': 'B0BLVNZNWV', 'quantity': 2}, {'product_id': 'B0BM4L3J9Q', 'quantity': 4}] to the shopping cart."

In [22]:
items_2 = [
    {
        "product_id": "B0B1BYBZRP",
        "quantity": 8,
    }
]

In [23]:
add_to_shopping_cart(items_2, "abc", "cde")

"Added [{'product_id': 'B0B1BYBZRP', 'quantity': 8}] to the shopping cart."

In [24]:
add_to_shopping_cart(items, "afdasfsdf", "dfgdfhdhdg")

"Added [{'product_id': 'B0BLVNZNWV', 'quantity': 2}, {'product_id': 'B0BM4L3J9Q', 'quantity': 4}] to the shopping cart."

### Get the Shopping Cart Items Tool

In [25]:
def get_shopping_cart(user_id: str, cart_id: str) -> list[dict]:

    """
    Retrieve all items in a user's shopping cart.
    
    Args:
        user_id: User ID
        cart_id: Cart identifier
    
    Returns:
        List of dictionaries containing cart items
    """
    
    conn = psycopg2.connect(
        host="localhost",
        port=5433,
        database="tools_database",
        user="langgraph_user",
        password="langgraph_password"
    )
    conn.autocommit = True

    with conn.cursor(cursor_factory=RealDictCursor) as cursor:

        query = """
                SELECT 
                    product_id, price, quantity,
                    currency, product_image_url,
                    (price * quantity) as total_price
                FROM shopping_carts.shopping_cart_items 
                WHERE user_id = %s AND shopping_cart_id = %s
                ORDER BY added_at DESC
            """
        cursor.execute(query, (user_id, cart_id))

        return [dict(row) for row in cursor.fetchall()]

In [26]:
get_shopping_cart("abc", "cde")

[{'product_id': 'B0B1BYBZRP',
  'price': Decimal('8.99'),
  'quantity': 8,
  'currency': 'USD',
  'product_image_url': 'https://m.media-amazon.com/images/I/31JF3Yk0GDL._AC_.jpg',
  'total_price': Decimal('71.92')},
 {'product_id': 'B0BM4L3J9Q',
  'price': Decimal('59.99'),
  'quantity': 4,
  'currency': 'USD',
  'product_image_url': 'https://m.media-amazon.com/images/I/51vDfmLEhsL._AC_.jpg',
  'total_price': Decimal('239.96')},
 {'product_id': 'B0BLVNZNWV',
  'price': Decimal('117.21'),
  'quantity': 2,
  'currency': 'USD',
  'product_image_url': 'https://m.media-amazon.com/images/I/41nxTtCkjyL._AC_.jpg',
  'total_price': Decimal('234.42')}]

In [27]:
get_shopping_cart("afdasfsdf", "dfgdfhdhdg")

[{'product_id': 'B0BM4L3J9Q',
  'price': Decimal('59.99'),
  'quantity': 4,
  'currency': 'USD',
  'product_image_url': 'https://m.media-amazon.com/images/I/51vDfmLEhsL._AC_.jpg',
  'total_price': Decimal('239.96')},
 {'product_id': 'B0BLVNZNWV',
  'price': Decimal('117.21'),
  'quantity': 2,
  'currency': 'USD',
  'product_image_url': 'https://m.media-amazon.com/images/I/41nxTtCkjyL._AC_.jpg',
  'total_price': Decimal('234.42')}]

In [28]:
get_shopping_cart("afdasfsdfsdfsdf", "dfgdfhdhdgsdfsdf")

[]

### Deleting Items from the Shopping Cart Tool

In [29]:
def remove_from_cart(product_id: str, user_id: str, cart_id: str) -> str:

    """
    Remove an item completely from the shopping cart.
    
    Args:
        user_id: User ID
        product_id: Product ID to remove
        cart_id: Cart identifier
    
    Returns:
        True if item was removed, False if item wasn't found
    """
    
    conn = psycopg2.connect(
        host="localhost",
        port=5433,
        database="tools_database",
        user="langgraph_user",
        password="langgraph_password"
    )
    conn.autocommit = True

    with conn.cursor(cursor_factory=RealDictCursor) as cursor:

        query = """
                DELETE FROM shopping_carts.shopping_cart_items
                WHERE user_id = %s AND shopping_cart_id = %s AND product_id = %s
            """
        cursor.execute(query, (user_id, cart_id, product_id))

        return cursor.rowcount > 0

In [31]:
remove_from_cart("B0BM4L3J9Q", "abc", "cde")

True

In [32]:
remove_from_cart("B0BM4L3J9Q", "abc", "cde")

False